Arbitrary N x N optical signal processor 

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import copy

# add angler to path (not necessary if pip installed)
import sys
sys.path.append("..")

# import the main simulation and optimization classes
from angler import Simulation, Optimization

# import some structure generators
from angler.structures import three_port, two_port, N_port, N_IO_port

import seaborn as sns 

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# LaTeX plot formatting
#Matplotlib params for figures
params = {"ytick.color" : "black",
          "xtick.color" : "black",
          "axes.labelcolor" : "black",
          "axes.edgecolor" : "black",
          "text.usetex" : True,}
plt.rcParams.update(params)
plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage[cm]{sfmath}')
sns.set_context("talk",font_scale=1)


Define simulation constants

In [ ]:
lambda0 = 1.55e-6           # free space wavelength (m)
c0 = 3e8                    # speed of light in vacuum (m/s)
omega = 2*np.pi*c0/lambda0  # angular frequency (2pi/s)
dl = 0.05                   # grid size (L0)
NPML = [20, 20]             # number of pml grid points on x and y borders
pol = 'Ez'                  # polarization (either 'Hz' or 'Ez')
source_amp = 1e-3           # mW? amplitude of modal source (make around 1 for nonlinear effects)

In [ ]:
# define material constants
n_index = 2.8        # refractive index (2-D Si)
eps_m = n_index**2   # relative permittivity

Inputs = AE features MNIST, Outputs = MNIST classes 

In [ ]:
# geometric parameters for a N -> N port device
L = 20              # length of box (L0) = micron 
N = 10              # Num output ports 
H = 20              # height of box (L0)
w = .5              # width of waveguides (L0)
d = H/10            # distance between waveguides (L0)
l = 5               # length of waveguide from PML to box (L0)
spc = 3             # space between box and PML (L0)
width= int(H/13/dl) # Width of source 

Definition of Simulation Object 

In [ ]:
# define permittivity of three port system
eps_r, design_region = N_IO_port(N, L, H, w, d, l, spc, dl, NPML, eps_m)
(Nx, Ny) = eps_r.shape
nx, ny = int(Nx/2), int(Ny/2)            # halfway grid points

# make new simulation objects, 1 fwd and 1 adj
simulation_fwd = Simulation(omega, eps_r, dl, NPML, pol)
simulation_adj = Simulation(omega, eps_r, dl, NPML, pol)

print("Computed a domain with {} grids in x and {} grids in y".format(Nx,Ny))
print("The simulation has {} grids per free space wavelength".format(int(lambda0/dl/simulation_fwd.L0)))

# plot the permittivity distribution
simulation_fwd.plt_eps(outline=False)
plt.show()

Sample input source 

In [ ]:
# Setup mode source params
src_pos_fwd = []
src_pos_adj = []

for i in range(N):
    center_fwd = [NPML[0]+int(l/2/dl), ny-((float(i)-float(N-1)/2.0)*d/dl + w)]
    center_adj = [-NPML[0]-int(l/2/dl), ny-((float(i)-float(N-1)/2.0)*d/dl + w)]
    # set the input waveguide modal source
    simulation_fwd.add_mode(neff=np.sqrt(eps_m), direction_normal='x', center=center_fwd, width=width, scale=source_amp)
    simulation_adj.add_mode(neff=np.sqrt(eps_m), direction_normal='x', center=center_adj, width=width, scale=source_amp)
    src_pos_fwd.append(center_fwd)
    src_pos_adj.append(center_adj)

srcs_fwd = simulation_fwd.setup_modes()
srcs_adj = simulation_adj.setup_modes()

Add modal profiles for auto-diff

In [ ]:
W_in = simulation_fwd.W_in   # input power (W/um)
print(W_in)

amp_test_list = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# TODO :: Write function "update_src" to encapsulate this ... 

# Update mode source amplitude ==> Forward Simulation 
for i in range(len(amp_test_list)):
    # Forward src
    center_fwd = src_pos_fwd[i]
    inds_x_fwd = [center_fwd[0], center_fwd[0]+1]
    inds_y_fwd = [int(center_fwd[1]-width/2), int(center_fwd[1]+width/2)]
    src = srcs_fwd[i].reshape((1, -1))*amp_test_list[i]
    simulation_fwd.src[inds_x_fwd[0]:inds_x_fwd[1], inds_y_fwd[0]:inds_y_fwd[1]] = np.abs(src[0])*np.sign(np.real(src[0]))


# Update mode source amplitude ==> Adjoint Simulation 
for i in range(len(amp_test_list)):
    center_adj = src_pos_adj[i]
    inds_x_adj = [center_adj[0], center_adj[0]+1]
    inds_y_adj = [int(center_adj[1]-width/2), int(center_adj[1]+width/2)]
    src = srcs_adj[i].reshape((1, -1))*amp_test_list[i]
    simulation_adj.src[inds_x_adj[0]:inds_x_adj[1], inds_y_adj[0]:inds_y_adj[1]] = np.abs(src[0])*np.sign(np.real(src[0]))

Simulate E-field propagation and plot

In [ ]:
# # set source and solve for electromagnetic fields
(Hx, Hy, Ez) = simulation_fwd.solve_fields()
(Hx, Hy, Ez) = simulation_adj.solve_fields()
simulation_adj.plt_abs(outline=True, cbar=True);
# plt.xlim((nx - 2*l*L), (nx + 2*l*L))
# plt.ylim((ny - 2*l*H), (ny + 2*l*H))
plt.show()
# print(np.real(Ez))


Design region 

In [ ]:
simulation_fwd.init_design_region(design_region, eps_m, style='halfway')
# Must ensure that design region is consistent between fwd and adj sims
# Fwd/Adj sim design region updated once every epoch 
simulation_adj.init_design_region(design_region, eps_m, style='halfway')
simulation_fwd.plt_eps(outline=False)
plt.show()

In [ ]:
(Hx, Hy, Ez) = simulation_fwd.solve_fields()
simulation_fwd.plt_abs(outline=True, cbar=True);
# plt.xlim((nx - 2*l*L), (nx + 2*l*L))
# plt.ylim((ny - 2*l*H), (ny + 2*l*H))
path = "../../Figures/sample_forward_sim.pdf"
plt.savefig(path, bbox_inches='tight')
plt.show()

Now measure power transmission of final simulation 

In [ ]:
# plot transmission characteristics
W_in = simulation_fwd.W_in   # input power (W/um)
print(W_in)
W_in = W_in
W_list = []
total_transmit = 0.0

for i in range(N):
    y_index = ny-((float(i)-float(N-1)/2.0)*d/dl + w)
    W_i_lin = simulation_fwd.flux_probe('x', center=[-NPML[0]-int(l/2/dl), y_index], width=width)
    W_list.append(W_i_lin)
    total_transmit += W_i_lin / W_in * 100
    print('        -> linear transmission (port {})        = {:.2f} %'.format(i, (W_i_lin / W_in * 100)))
print('total transmission = ', total_transmit)
print(W_list)